In [ ]:
import pandas as pd, ipywidgets as widgets, numpy as np, matplotlib.pyplot as plt, io
from adjustText import adjust_text
raw_uploader, eln_uploader = widgets.FileUpload(multiple = False), widgets.FileUpload(multiple = False)
display("Upload ELN w/ Reporter Data and Antibodies Sheet (xlsx)")
display(eln_uploader)

In [ ]:
elnname = list(eln_uploader.value.keys())[0]
eln = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']))
reporter_antagonist_data = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), sheet_name = "Reporter Antagonist")
ab_sheet = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), sheet_name = "Antibody Plate Map")
reporter_antagonist_data

In [ ]:
#Concatenate all antagonist columns into a their respective single Series
#The input for this code should be FFLuc divided by RLuc already (the divided table)
reporter_antagonist_column = pd.Series()
for column in reporter_antagonist_data.columns:
    colToAdd = reporter_antagonist_data[column]
    reporter_antagonist_column = pd.concat([reporter_antagonist_column, colToAdd], axis = 0, ignore_index = True)

In [ ]:
#Separate n=1 and n=2 duplicates, average them and reset the indeces: for Antagonist conditions
firsts = reporter_antagonist_column.iloc[::2]
seconds = reporter_antagonist_column.iloc[1::2]
first_reset = firsts.reset_index(drop = True)
second_reset = seconds.reset_index(drop = True)
averageAntagonists = first_reset + second_reset
averageAntagonists = averageAntagonists / 2

In [ ]:
#Create new DF for concatenating the Agonist, Antagonist signals, and the Antibody Plate Map
OutputSheet = pd.concat([averageAgonists, averageAntagonists, ab_sheet], axis = 1)
OutputSheet.index += 1
OutputSheet.rename(inplace = True, columns = {0: "Antagonist Mode"})
OutputSheetCleaned = OutputSheet[["Antagonist Mode", "DNA", "Concentration (mg/mL)"]]
OutputSheetCleaned.to_excel("Reporter_Output.xlsx")
AntagonistSort = OutputSheetCleaned.sort_values(by="Antagonist Mode", ascending = False)

In [ ]:
# Top10Antagonists = OutputSheetCleaned.nsmallest(10, "Antagonist Mode")
# fig, ax = plt.subplots()
# ax.scatter(x = OutputSheetCleaned["Protein Concentration (ug/uL)"], y=OutputSheetCleaned["Antagonist Mode"], alpha = .2, color = "red")
# ax.scatter(x = Top10Antagonists["Protein Concentration (ug/uL)"], y = Top10Antagonists["Antagonist Mode"], alpha = .8, color = "red", label = "Top Antagonist Hits")
# ax.set_title("BRET Primary Screen")
# ax.set_xlabel("Concentration (mg/ml)")
# ax.set_ylabel("cAMP Levels")
# ax.legend()
